# Data Cleaning and Preparation

### Import libraries

In [60]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapely as shapely
import geopandas as gpd

### Load Data and Keep Relevant Columns

In [61]:
# Data files
claims_data_file = 'FimaNfipClaims.parquet' #claims data file
metadata_file = 'fima-claims-metadata.csv' #metadata file

# Read metadata to select the features to use
metadata = pd.read_csv(metadata_file)
cols = metadata[metadata['Used in Model'] == 1]['Name'].tolist()

print(f"Columns used in Model:")
cols

Columns used in Model:


['basementEnclosureCrawlspaceType',
 'elevatedBuildingIndicator',
 'elevationDifference',
 'ratedFloodZone',
 'lowestFloorElevation',
 'numberOfFloorsInTheInsuredBuilding',
 'occupancyType',
 'originalConstructionDate',
 'yearOfLoss',
 'buildingDamageAmount',
 'buildingPropertyValue',
 'floodCharacteristicsIndicator',
 'floodWaterDuration',
 'waterDepth',
 'countyCode',
 'latitude',
 'longitude']


#### Load and preprocess claims data


##### Load claims data

In [62]:
# Load the claims data for the selected columns
claims_raw = pd.read_parquet(claims_data_file, columns = cols)

##### Drop rows where `buildingDamageAmount` and `basementEnclosureCrawlspaceType` are missing

In [63]:
# Drop rows with missing buildingDamageAmount
claims_raw.dropna(subset=['buildingDamageAmount'], inplace=True)
claims_raw = claims_raw[claims_raw['buildingPropertyValue'] > 0]

# Drop rows with missing basementEnclosureCrawlspaceType
claims_raw.dropna(subset=['basementEnclosureCrawlspaceType'], inplace=True)
claims = claims_raw.copy()

##### Create `damageRatio` and keep valid rows

In [64]:
# Create Damage Ratio column
claims['damageRatio'] = claims['buildingDamageAmount'] / claims['buildingPropertyValue']
claims.drop(columns=['buildingDamageAmount', 'buildingPropertyValue'], inplace=True)
claims.dropna(subset=['damageRatio'], inplace=True)

# Keep only claims with valid damage ratios
claims['damageRatio'] = claims[(claims['damageRatio'] >=0) & (claims['damageRatio'] <= 1)]['damageRatio']
claims.dropna(subset=['damageRatio'], inplace=True)

##### Clean `waterDepth`

In [65]:
# Some claims have water depth incorrectly in feet instead of inches
# We will only keep rows where water depth is > 6 assuming smaller values are in feet
# Drop outliers with depth > 180 inches
claims = claims[(claims['waterDepth'] > 6) & (claims['waterDepth'] < 180)].copy()

# Convert water depth from inches to feet
claims['waterDepth'] = claims['waterDepth']/12

#### Check features with most missing values 

In [66]:
def missing_value_percent(df):
    """
    Function to calculate the percentage of missing values in each column of a DataFrame.
    """
    return df.isnull().sum().sort_values(ascending=False)/len(claims)*100

# Percentage of missing values
missing_value_percent(claims)

floodCharacteristicsIndicator         99.214123
lowestFloorElevation                  76.569725
elevationDifference                   75.347814
floodWaterDuration                     5.627890
ratedFloodZone                         0.699684
countyCode                             0.468484
latitude                               0.284944
longitude                              0.284944
numberOfFloorsInTheInsuredBuilding     0.030421
occupancyType                          0.004056
basementEnclosureCrawlspaceType        0.000000
elevatedBuildingIndicator              0.000000
originalConstructionDate               0.000000
yearOfLoss                             0.000000
waterDepth                             0.000000
damageRatio                            0.000000
dtype: float64

#### Keep columns with missing values < 60%

In [67]:
keep_cols = [col for col in claims.columns if claims[col].isnull().sum() < 0.6*len(claims)]
claims = claims[keep_cols].copy()

print("Columns kept after removing those with more than 60% missing values:")
keep_cols

Columns kept after removing those with more than 60% missing values:


['basementEnclosureCrawlspaceType',
 'elevatedBuildingIndicator',
 'ratedFloodZone',
 'numberOfFloorsInTheInsuredBuilding',
 'occupancyType',
 'originalConstructionDate',
 'yearOfLoss',
 'floodWaterDuration',
 'waterDepth',
 'countyCode',
 'latitude',
 'longitude',
 'damageRatio']

### Create derived features

#### Compute `buildingAgeAtLoss`

In [68]:
# Convert originalConstructionDate to year except for null values
claims['originalConstructionDate'] = (claims['originalConstructionDate']
                                        .apply(lambda x: x.year if not pd.isna(x) else x))

# Convert all 1492 construction year to NaN
claims['originalConstructionDate'] = (claims['originalConstructionDate'].replace(1492, np.nan))

# Fill null values with median construction date
median_construction_date = claims['originalConstructionDate'].median()
claims['originalConstructionDate'] = pd.to_numeric(claims['originalConstructionDate'].fillna(median_construction_date))

# Use OriginalConstructionDate and yearOfLoss to calculate buildingAgeAtLoss
claims["buildingAgeAtLoss"] = claims['yearOfLoss'] - claims['originalConstructionDate']
claims['buildingAgeAtLoss'] = claims['buildingAgeAtLoss'].clip(lower=1)

# Drop originalConstructionDate column but keep yearOfLoss column as it may be a useful predictor
claims.drop(columns=['originalConstructionDate'], inplace=True)

#### Fix Data Types

In [69]:
claims['countyCode'] = claims['countyCode'].astype(str)

### Handle Unknowns

In [70]:
def impute_missing_values(df: pd.DataFrame, col: str, by: str | list, method: str):
    """
    Function to impute missing values in a DataFrame.
    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        col (str): The column with missing values to be imputed.
        by (str | list): The column(s) to group by for imputation.
        method (str): The method to use for imputation ('median', 'mean', 'mode').
    Returns:
        pd.Series: A Series with the imputed values for the specified column.
    """

    #assert df[col].isnull().sum() > 0, f"Column {col} has no missing values"
    assert method in ['median', 'mean', 'mode'], f"Method {method} is not supported"
    
    # Create a DataFrame with only rows where the column is not null
    known_df = df.dropna(subset=col)
    initial_unknowns = df[col].isna().sum()

    # Calculate the imputation map based on the specified method
    if method == 'median':
        impute_map = known_df.groupby(by)[col].median()  
    elif method == 'mean':
        impute_map = known_df.groupby(by)[col].mean()  
    elif method == 'mode':
        impute_map = known_df.groupby(by)[col].agg(lambda x: x.mode()[0])  

    # Define a function to impute missing values based on the group
    def impute_by_group(row):
        if pd.notna(row[col]):  
            return row[col]
        group_key = tuple(row[by]) if isinstance(by, list) else row[by]  
        return impute_map.get(group_key, np.nan)  

    # Apply the imputation function to each row and extract the imputed column
    print(f'Imputing {col} by {by} {method}')
    imputed_column = df.apply(impute_by_group, axis=1)
    unknowns_after_targeted_imputation = imputed_column.isna().sum()
    print(f"\t{initial_unknowns - unknowns_after_targeted_imputation} unknowns were filled with targeted imputation")

    # For remaining unfilled rows, fill by the 'method' over all rows
    if method == 'median':
        fill_value = imputed_column.median()
    elif method == 'mean':
        fill_value = imputed_column.mean()
    elif method == 'mode':
        fill_value = imputed_column.mode()[0]
    
    imputed_column = imputed_column.fillna(fill_value)
    final_unknowns = imputed_column.isna().sum()
    print(f'\t{unknowns_after_targeted_imputation- final_unknowns} were filled by overall {method} ({fill_value})')
    print(f'\t{final_unknowns} unknowns left\n')

    return imputed_column, impute_map

##### Drop unknown `ratedFloodZone` and `countyCode`

In [71]:
# Drop unkown rateFloodZone
claims.dropna(subset = ['ratedFloodZone', 'countyCode', 'waterDepth', 'floodWaterDuration'], inplace=True)

##### Impute missing values for `occupancyType` and `numberOfFloorsInTheInsuredBuilding`

In [72]:
claims['occupancyType'] = impute_missing_values(claims, 'occupancyType', 'countyCode', 'mode')[0]
claims['numberOfFloorsInTheInsuredBuilding'] = impute_missing_values(claims, 'numberOfFloorsInTheInsuredBuilding', ['occupancyType', 'countyCode'],'median')[0]

Imputing occupancyType by countyCode mode
	2 unknowns were filled with targeted imputation
	0 were filled by overall mode (1.0)
	0 unknowns left

Imputing numberOfFloorsInTheInsuredBuilding by ['occupancyType', 'countyCode'] median
	3 unknowns were filled with targeted imputation
	0 were filled by overall median (2.0)
	0 unknowns left



### Code categorical columns

In [73]:
# List categorical columns
categorical_cols = ['ratedFloodZone', 'occupancyType', 'basementEnclosureCrawlspaceType',
                    'countyCode', 'elevatedBuildingIndicator']
# List numerical columns
numerical_cols = [col for col in claims.columns if col not in categorical_cols]

In [74]:
all_categories = {}
for col in categorical_cols:
    categories = claims[col].to_list()
    categorical_col = pd.Categorical(claims[col])
    codes = categorical_col.codes
    category_map = list(set([(cat,code) for cat, code in zip(categories, codes)]))
    category_map = {cat:code for cat, code in category_map}
    all_categories.update({col: category_map})
    claims[col] = codes

### Final Data

#### Save cleaned claims data

In [76]:
claims.drop(columns = ['latitude', 'longitude', 'yearOfLoss']).to_csv('claims_cleaned.csv', index = False)

#### Plot the geographical extent of data

In [77]:
gdf = gpd.GeoDataFrame(claims[['longitude', 'latitude', 'damageRatio', 'waterDepth']].sample(10_000, random_state=42))
gdf = gdf.set_geometry(gpd.points_from_xy(gdf.longitude, gdf.latitude)).set_crs(epsg=4326).drop(columns=['longitude', 'latitude'])
gdf.explore(column = 'damageRatio', cmap='Spectral_r', legend=True, tiles='CartoDB positron')